In [ ]:
# Generate Spark Session
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .appName("Reading from Sockets")
    .config("spark.sql.streaming.forceDeleteTempCheckpointLocation", True)
    .master("spark://spark-master:7077")
    .getOrCreate()
)

spark

In [11]:
spark.conf.set("spark.sql.streaming.forceDeleteTempCheckpointLocation", True)

# AQE DOES NOT WORK FOR STREAMING APPLICATIONS
spark.conf.set("spark.sql.adaptive.enabled", False)
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled", False)

# SEE DIFFERENCE IN PROCESSING TIME BY CHANGING THIS PARAMETER
spark.conf.set("spark.sql.shuffle.partitions", 1)

In [ ]:
# IN A TERMINAL OF JUPYTER
# ncat -l 9999
# write lines of text + ENTER so spark receives them through the socket connection

In [12]:
# Read input data

df_raw = spark.readStream.format("socket").option("host","localhost").option("port", "9999").load()

24/11/16 11:52:47 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [13]:
df_raw.printSchema()

root
 |-- value: string (nullable = true)



In [14]:
# Split the line into words
from pyspark.sql import functions as F

df_words = df_raw.withColumn("words", F.split("value", " "))
df_explode = df_words.withColumn("word", F.explode("words")).drop("value", "words")
df_agg = df_explode.groupBy("word").count()

In [16]:
# Write the output to console streaming

df_agg.writeStream.format("console").outputMode("complete").start().awaitTermination()

24/11/16 11:53:15 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-40a6e7c6-38f2-4ebf-ab7e-93dd540da086. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+

-------------------------------------------
Batch: 1
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|mundo|    1|
| hola|    1|
+-----+-----+



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [18]:
spark.stop()